In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
pkg_path = "/nfs/homedirs/yuny/project-1/audio_classification"
if pkg_path not in sys.path:
    sys.path.append(pkg_path)
    
import yaml
from audio_classification.tools import do_train
from audio_classification.data import BMWDataset

In [2]:
with open("/nfs/students/winter-term-2020/project-1/project-1/audio_classification/configs/m11_bmw.yaml", "r") as config_file:
    configs = yaml.load(config_file)

In [ ]:
# Use a loop to train with k fold_cv
base_save_name = configs['CHECKPOINT']['SAVE_NAME']
for i in range(1, 11): # Training using validation fold 1, 2, ...., 10
    print("Start training with validation fold {}".format(i))
    configs['DATASET']['VAL_FOLD']=i
    configs['CHECKPOINT']['SAVE_NAME'] = base_save_name + '-fold' + str(i)
    do_train(configs)

Start training with validation fold 1


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 3 K   
